Extract course schedule information from Austin Community College's website and write to a google sheet with date and time stamps.

In [43]:
import requests
import time 
import re

In [44]:
from bs4 import BeautifulSoup

Beautiful Soup: python library that allows you to play with the html code on a website in order to extract the information that you are interested in.  https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [45]:
# Import from ACC website into beautiful soup
url = 'http://www6.austincc.edu/schedule/'
response= requests.get(url)
html = response.content
soup = BeautifulSoup(html)
# Run the next command if you want to see what beautiful soup did to the html code
#print(soup.prettify())


In [46]:
# Get URL for disciplines in "disciplinelist"
#disciplinelist = ["Chemistry","Astronomy","Physics", "Biology","Engineering","Economics"]
disciplinelist = ["Astronomy","Physics"]
discipline=soup.find_all('a', string=disciplinelist)
urls = []
disciplines = []
for index, i in enumerate(discipline):
    urltemp = "http://www6.austincc.edu/" + discipline[index]['href'] 
    disciptemp = discipline[index].string
    urls.append(urltemp)
    disciplines.append(disciptemp)
#print disciplines, urls

In [47]:
# Allow python to write to google sheets. Will need to be updated soon.
# For more info, see links: https://developers.google.com/google-apps/spreadsheets/
# Follow instruction in links below to create .json file needed below "filename"
# http://gspread.readthedocs.io/en/latest/oauth2.html#oauth-credentials 
import json
import gspread
from oauth2client.client import SignedJwtAssertionCredentials
####  on next line replace 'filename' with path and filename of .json file  which contains private key #####
json_key = json.load(open('/Users/Andrea/Projects/WorkProjects/CourseSchedules/Course Schedule-7d7036b277cb.json'))
#json_key = json.load(open('filename'))
scope = ['https://spreadsheets.google.com/feeds']
credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'].encode(),scope)
gc = gspread.authorize(credentials)
wks = gc.open("ACC")

In [48]:
# Import html from each discipline URL's into beautiful soup

for index, URLx in enumerate(urls):
    print disciplines[index], urls[index]
    response = requests.get(URLx)
    html = response.content
    soup = BeautifulSoup(html)
#    print soup.prettify
    list_of_rows=[]
    timenow = (time.strftime(" %H:%M:%S %m/%d/%Y"))
    
    term = soup.find_all(text=re.compile('Fall 2016'), limit=1)
    print term
    print str(term)
    print term[0].string
    
    title = disciplines[index] + str(timenow)
    print title
    
    for row in soup.find_all('table','section_line'):
        list_of_cells=[]
#        print list_of_cells
        for cell in row.find_all("td"):
            texta=cell.text
            list_of_cells.append(texta)
#            print "texta",texta
            textb = cell.find_previous('h4')
        textc = textb.text
        list_of_cells.append(textc)
        list_of_rows.append(list_of_cells)
    length = len(list_of_rows)+1
    print length
#Create new google worksheet within google sheet for each discipline with date/time included in name
    worksheet = wks.add_worksheet(title, rows=length,cols='3')
    worksheet.update_cell(1,1,urls[index]) 
    worksheet.update_cell(1,2,term[0].string)
    for index, row in enumerate(list_of_rows, start=2):
        worksheet.update_cell(index,1,row[16])
        worksheet.update_cell(index,2,row[5])
        worksheet.update_cell(index,3,row[15])
        

Astronomy http://www6.austincc.edu//schedule/index.php?op=browse&opclass=ViewSched&term=216F000&disciplineid=TFPHA&yr=2017&ct=CC
[u'Course Schedule by Discipline - (Credit Fall 2016) ']
[u'Course Schedule by Discipline - (Credit Fall 2016) ']
Course Schedule by Discipline - (Credit Fall 2016) 
Astronomy 21:55:57 06/23/2016
19
Physics http://www6.austincc.edu//schedule/index.php?op=browse&opclass=ViewSched&term=216F000&disciplineid=TFPHP&yr=2017&ct=CC
[u'Course Schedule by Discipline - (Credit Fall 2016) ']
[u'Course Schedule by Discipline - (Credit Fall 2016) ']
Course Schedule by Discipline - (Credit Fall 2016) 
Physics 21:56:24 06/23/2016
91
